# Why `cmomy`?

$$\newcommand{\ave}[1]{\langle{#1}\rangle}$$

We wish to calculate moments of the form $\ave{x}$ and $\ave{(\delta x)^k}$, $\delta x = (x - \ave{x})$, where, in general we define averages
of the form.
$$
   \ave{x} = \frac{\sum_i w_i x_i}{\sum_i w_i}
$$
$$
  \ave{(\delta x)^k} = \frac{\sum_i w_i (x_i - \ave{x})^k}{\sum_i w_i}
$$
where $w_i$ are weights associated with each sample $x_i$.  In the simple case, $w_i = 1$.  To calculate a central moment with $k > 1$, we have some choices.  

1. First calculate $\ave{x}$, then use the above formula.
* Downside: requires two passes through the data.  
* Upside: numerically stable.
    
2. Expand out the central moment to raw moments.  For example $\ave{(x - \ave{x})^2} = \ave{x^n} - \ave{x}^2$.  These raw moments can be calculated directly in one pass.  
* Downside: numerically unstable ([see here for some info](https://en.wikipedia.org/wiki/Algorithms_for_calculating_variance)). 
* Upside: single pass for each of $\ave{x^k}$.


## An example

    

In what follows, we'll arrange data in the from `output[moment]` where `output[0] = {total weight}`, `output[1] = {average value}`, `output[k > 1] = {kth order central moment}`.  We can generate the central moments using the following functions

In [1]:
def cmom_1(x, axis=0, mom=3):
    """
    create central moments using stable method
    """
    # output shape
    shape = x.shape[:axis] + x.shape[axis+1:] + (mom+1,)
    output = np.zeros(shape, dtype=x.dtype)

    output[..., 0] = x.shape[axis]
    output[..., 1] = x.mean(axis=axis)
    
    # moments [2 -> mom]
    output[..., 2:] = ((x - x.mean(axis=axis, keepdims=True))[..., None] ** np.arange(2, mom+1)).mean(axis=axis)
    return output


def cmom_2(x, axis=0, mom=3):
    
    if mom > 3:
        raise ValueError
        
    shape = x.shape[:axis] + x.shape[axis+1:] + (mom+1,)
    
    output = np.zeros(shape, dtype=x.dtype)
    
    output[..., 0] = x.shape[axis]

    # higher order raw moments
    raws = (x[..., None]**np.arange(0, mom+1)).mean(axis=axis)
    
    output[..., 1] = raws[..., 1]
    if mom > 1:
        # <(x - <x>)**2> = <x**2> - <x>**2
        output[..., 2] = raws[..., 2] - raws[..., 1] ** 2
        
    if mom > 2:
        # <(x - <x>)**3> = <x**3> - 3<x**2><x> + 2<x>**3
        output[..., 3] = raws[..., 3] - 3 * raws[..., 2] * raws[..., 1] + 2 * raws[..., 1]**3
    
    return output


In [2]:
import numpy as np
np.random.seed(0)
n = 1000
x = np.random.rand(n)

m1 = cmom_1(x)
m2 = cmom_2(x)
# same values
np.testing.assert_allclose(m1, m2)

OK, great.  But what if we have unscaled data?  For example, setting $y = a x + b$ gives $\ave{y} = a \ave{x} + b$, but for central moments, $\ave{(y - \ave{y})^n} = \ave{(a x - b - \ave{a x - b})^n} = a^n \ave{(\delta x)^n}$

In [3]:
# large value, small fluctuation
a = 0.1
b = 1e4
y = a * x + b

# expected value after shift
expected = m1.copy()
expected[..., 1] = a * expected[..., 1] + b
expected[..., 2:] = expected[..., 2:] * (a ** np.arange(2, 4))


m1_shift = cmom_1(y)
m2_shift = cmom_2(y)

m1_error = m1_shift - expected
m2_error = m2_shift - expected

print('abs error using central moments:', m1_error)
print('rel error using central moments:', m1_error / expected)
print()
print('abs error using raw moments    :', m2_error)
print('rel error using raw moments    :', m2_error / expected)


abs error using central moments: [0.00000000e+00 0.00000000e+00 3.98444298e-16 1.46035150e-15]
rel error using central moments: [0.00000000e+00 0.00000000e+00 4.71823816e-13 9.33279122e-10]

abs error using raw moments    : [ 0.00000000e+00  1.81898940e-12  4.64434976e-08 -2.68711163e-03]
rel error using raw moments    : [ 0.00000000e+00  1.81898038e-16  5.49967672e-05 -1.71727503e+03]


## So what?

OK, so central moments have some advantages.  But the two pass thing makes life difficult.  For example, what if we are running a slow experiment or simulation?  We could collect all the samples for `x`, then do the two pass algorithm at the end.  But it would be nice to calculate averages on the fly.  This is why we often turn to raw moments.  There easy to accumulate as data comes in, while central moments, at least as formulated above, are not.  

Thankfully, smart people have figured out how to calculate central moments in a one-pass way.  Moreover, these methods allow central moments to be combined.  This makes resampling from central moments possible.  We don't go into detail on these formulas, but point the interested reader to the this [article](https://en.wikipedia.org/wiki/Algorithms_for_calculating_variance), and citations within.

The `cmomy` package has a variety of routines to work with central moments.  This include

* Create central moments accumulator object in a variety of ways
* add data to object
* resample data
* extract central and raw moments


In [4]:
import cmomy

cx = cmomy.CentralMoments.from_vals(x, axis=0, mom=3)
cy = cmomy.CentralMoments.from_vals(y, axis=0, mom=3)


print(cx)

# access underlying data
print(cx.values)

<CentralMoments(val_shape=(), mom=(3,))>
array([1.00000000e+03, 4.95921534e-01, 8.44476867e-02, 1.56475321e-03])
[1.00000000e+03 4.95921534e-01 8.44476867e-02 1.56475321e-03]


Basically, the `cmomy.CentralMoments` object is just a collection of routines around `data`, which is an array of central moments (with the convension thata `data[..., 0]` is the total `weight`, `data[..., 1]` is the average, and `data[..., k > 1]` is the `kth` central moment).  It gives the same results as calculating central moments directly:

In [5]:
np.testing.assert_allclose(cx.values, m1)
np.testing.assert_allclose(cy.values, m1_shift)

Nice!  But what can that's not all that special.  The real power comes in for multidimensional data.
Suppose that we have separate samples of the data $\{x_a, x_b, ... \}$, and want to get there central moments.  We can do the following.

In [6]:
x = np.random.rand(100, 5)
m1 = cmom_1(x, axis=0, mom=4)
m1

array([[ 1.00000000e+02,  4.74853227e-01,  8.47244951e-02,
         2.76974293e-03,  1.34068030e-02],
       [ 1.00000000e+02,  4.80722749e-01,  8.78818646e-02,
         8.19341633e-04,  1.39987970e-02],
       [ 1.00000000e+02,  4.90766371e-01,  9.23693311e-02,
         5.53132226e-03,  1.52942205e-02],
       [ 1.00000000e+02,  5.43783131e-01,  1.02492232e-01,
        -5.35859195e-03,  1.63960397e-02],
       [ 1.00000000e+02,  5.66765762e-01,  8.06207435e-02,
        -8.46362454e-03,  1.33037607e-02]])

Wrapping the data in a central moments object gives

In [7]:
c = cmomy.CentralMoments.from_data(m1, mom_ndim=1)

# reduce the data along a dimension to get

print(c.reduce(axis=0).values)

# verify that this is the same as just accumulating all the data
print(cmom_1(x.reshape(-1), mom=4))

[ 5.00000000e+02  5.11378248e-01  9.09810411e-02 -8.60371890e-04
  1.44841954e-02]
[ 5.00000000e+02  5.11378248e-01  9.09810411e-02 -8.60371890e-04
  1.44841954e-02]


Excellent! We can use `cmomy.CentralMoments` to accumulate data on the go

In [8]:
c = cmomy.CentralMoments.zeros(mom=4)
for i in range(5):
    # note: pushing multiple values
    c.push_vals(x[:, i])
print(c.values)

# or, you can push all the values one at a time!
c.zero()
for xx in x.reshape(-1):
    # note: pushing single value
    c.push_val(xx)
    
print(c.values)


[ 5.00000000e+02  5.11378248e-01  9.09810411e-02 -8.60371890e-04
  1.44841954e-02]
[ 5.00000000e+02  5.11378248e-01  9.09810411e-02 -8.60371890e-04
  1.44841954e-02]


The values can even be differently weighted.

In [9]:
xs = np.split(x.reshape(-1), [50, 150, 300])

c.zero()
for v in xs:
    c.push_vals(v)
    
print(c.values)

#Alternatively, you can add objects together
cs = [cmomy.CentralMoments.from_vals(v, mom=4) for v in xs]
print(cs)

c_tot = cs[0] + cs[1] + cs[2] + cs[3]
print(c_tot.values)

# or more simply
print(sum(cs, start=cs[0].zeros_like()).values)

[ 5.00000000e+02  5.11378248e-01  9.09810411e-02 -8.60371890e-04
  1.44841954e-02]
[<CentralMoments(val_shape=(), mom=(4,))>
array([5.00000000e+01, 4.83222666e-01, 8.91597412e-02, 1.70393418e-03,
       1.49340794e-02]), <CentralMoments(val_shape=(), mom=(4,))>
array([1.00000000e+02, 4.89588725e-01, 9.60199692e-02, 1.91605463e-03,
       1.59749766e-02]), <CentralMoments(val_shape=(), mom=(4,))>
array([ 1.50000000e+02,  5.22174233e-01,  9.16583598e-02, -1.91189645e-03,
        1.45006794e-02]), <CentralMoments(val_shape=(), mom=(4,))>
array([ 2.00000000e+02,  5.21214916e-01,  8.77891621e-02, -1.88818756e-03,
        1.35395457e-02])]
[ 5.00000000e+02  5.11378248e-01  9.09810411e-02 -8.60371890e-04
  1.44841954e-02]
[ 5.00000000e+02  5.11378248e-01  9.09810411e-02 -8.60371890e-04
  1.44841954e-02]


Very cool!

## weighted averages

cmomy is setup to work with arbitrary weights.  We saw this work above when we considered unequal sample sizes.  For example,

In [10]:
def get_cmom_with_weights(x, w, axis=0, mom=3):
    """create central moments using stable method"""
    # output shape
    shape = x.shape[:axis] + x.shape[axis+1:] + (mom+1,)
    output = np.zeros(shape, dtype=x.dtype)

    w_sum = w.sum(axis=axis)
    w_norm = 1. / w_sum
    
    output[..., 0] = w_sum
    output[..., 1] = ((w*x).sum(axis=axis)) * w_norm
    
    
    # moments [2 -> mom]
    output[..., 2:] = (
        w[..., None] * (x - (w * x).sum(axis=axis, keepdims=True) * w_norm)[..., None] ** np.arange(2, mom+1)
    ).sum(axis=axis) * w_norm

    return output

In [11]:
x = np.random.rand(100)
w = np.random.rand(100)

In [12]:
moments = get_cmom_with_weights(x, w)
moments

array([ 5.13374609e+01,  5.44154373e-01,  8.92382985e-02, -6.97037369e-03])

In [13]:
cmomy.CentralMoments.from_vals(x, w=w, mom=3)

<CentralMoments(val_shape=(), mom=(3,))>
array([ 5.13374609e+01,  5.44154373e-01,  8.92382985e-02, -6.97037369e-03])

## comoments

cmomy can alos handle comoments (covariance, etc), like the $\ave{(\delta x)^i (\delta y)^j}$    
For example

In [14]:
y = np.random.rand(100)

cmomy.CentralMoments.from_vals(x=(x, y), w=w, mom=(2,2))

<CentralMoments(val_shape=(), mom=(2, 2))>
array([[ 5.13374609e+01,  4.71249550e-01,  9.15677703e-02],
       [ 5.44154373e-01,  1.80645670e-02, -2.01103696e-03],
       [ 8.92382985e-02, -3.58022115e-03,  7.03049577e-03]])

All the special sauce works for central moments or comoments.  

## xarray DataArray support

cmomy also works with DataArray objects.  For example

In [15]:
import xarray as xr

x = xr.DataArray(np.random.rand(100, 2, 3), dims=['rec','a','b'])
x.head(3)

<xarray.DataArray (rec: 3, a: 2, b: 3)>
array([[[0.54695006, 0.29361682, 0.96820439],
        [0.2261963 , 0.01573824, 0.32585484]],

       [[0.5025094 , 0.02836293, 0.5592483 ],
        [0.87428277, 0.70473219, 0.62296832]],

       [[0.95596175, 0.95827933, 0.82426647],
        [0.60774185, 0.48776456, 0.01331611]]])
Dimensions without coordinates: rec, a, b

To create a cmomy object wrapping a DataArray, use `cmomy.xCentralMoments`

In [16]:
c = cmomy.xCentralMoments.from_vals(x, dim='rec', mom=3)
c

<xCentralMoments(val_shape=(2, 3), mom=(3,))>
<xarray.DataArray (a: 2, b: 3, mom_0: 4)>
array([[[ 1.00000000e+02,  5.36721173e-01,  8.32281081e-02,
         -2.27158777e-03],
        [ 1.00000000e+02,  4.94492143e-01,  7.06976276e-02,
          2.51295615e-03],
        [ 1.00000000e+02,  5.47831197e-01,  7.45661739e-02,
         -6.10210425e-03]],

       [[ 1.00000000e+02,  5.07077442e-01,  8.76007027e-02,
         -8.65285379e-04],
        [ 1.00000000e+02,  4.60226078e-01,  9.39251917e-02,
          5.82696382e-03],
        [ 1.00000000e+02,  4.76483437e-01,  7.99380896e-02,
          5.50961784e-03]]])
Dimensions without coordinates: a, b, mom_0

Everything that `CentralMoments` can do, `xCentralMoments` can do.